Modelar o ciclo de cogeração ilustrado abaixo. Permitir que o processo varie de 
0% (desligado) até 100% (completamente ligado).

![Ciclo](img/ciclo.jpeg "Ciclo")

Importando bibliotecas necessárias

In [100]:
from pandas import read_excel, DataFrame
from CoolProp.CoolProp import PropsSI as prop
from CoolProp.CoolProp import State as st
from numpy import arange, absolute, nan

In [101]:
df = read_excel('data.xlsx')
display(df.replace(nan, '-'))

,Temperatura [ºC],Pressão [kPa],Vazão [kg/s],Título [-]
0,485,6495,27.9,-
1,-,900,-,-
2,-,250,-,-
3,51,Saturação,-,-
4,51,Saturação,-,0.0
5,-,250,-,-
6,110,250,-,-
7,-,6495,27.9,-
8,110,250,-,-
9,-,900,-,-


Dados fornecidos

In [102]:
fluido = 'Water'
n_turb, n_pump = 0.85, 0.85

Inserindo temperatura de saturação na tabela e convertendo para K

In [103]:
T, c = [], 0

for i in df['Temperatura [ºC]']:
    c = c + 1
    try:
        if i == 'Saturação':
            T.insert(c-1, round(prop('T', 'P', df['Pressão [kPa]'][c-1], 'Q', 0, fluido), 2) )
        else:
            T.insert(c-1, i + 273.15)
    except ValueError or NameError:
        print(f'Sem informações de pressão no estado {c}')

Inserindo pressão de saturação na tabela 

In [104]:
P, c = [], 0
for i in df['Pressão [kPa]']:
    c = c + 1
    try:
        if i == 'Saturação':
            P.insert(c-1, round(prop('P', 'T', T[c-1], 'Q', 0, fluido),2) )
        else:
            P.insert(c-1, i)
    except ValueError or NameError:
        print(f'Sem informações de temperatura no estado {c}')

Definindo valores de vazão e titulo a partir da tabela de dados

In [105]:
V = df['Vazão [kg/s]']
x = df['Título [-]']

## Definindo estados ##

Definindo estados em função da pressão e temperatura com dados fornecidos

$$ st: Estado $$
$$st: st(P, T)$$

In [106]:
st1 = st(fluido, {'P': P[0], 'T': T[0] })
st4 = st(fluido, {'P': P[3], 'T': T[3] })
st7 = st(fluido, {'P': P[6], 'T': T[6] })
st9 = st(fluido, {'P': P[8], 'T': T[8]})
st11 = st(fluido, {'P': P[10], 'T': T[10]})

Definindo estados em função da pressão e titulo com dados fornecidos

$$st: st(P, x)$$

In [107]:
st5 = st(fluido, {'P': P[4], 'Q': x[4] })
st12 = st(fluido, {'P': P[11], 'Q': x[11]})

Definindo estados a partir da equação de eficiência isentrópica para turbinas

$$ \eta_{turb} = \frac{h_{n-1} - h_{n}}{h_{n-1} - h_{ns}} $$

Estado 2

$$ st_{2s}: st_{2s}(P_{2}, S_{1}) $$
$$ h_{2} = h_{1} - \eta_{turb}(h_{1} - h_{2s}) $$
$$ st_{2}: st_{2}(P_{2}, h_{2}) $$

In [108]:
st2s = st(fluido, {'P': P[1], 'S': st1.s})
h2 = st1.h - n_turb*(st1.h - st2s.h)
st2 = st(fluido, {'P': P[1], 'H': h2})

Estado 3

$$ st_{3s}: st_{3s}(P_{3}, S_{2}) $$
$$ h_{3} = h_{2} - \eta_{turb}(h_{2} - h_{3s}) $$
$$ st_{3}: st_{2}(P_{3}, h_{3}) $$

In [109]:
st3s = st(fluido, {'P': P[2], 'S': st2.s })
h3 = st2.h - n_turb*(st2.h - st3s.h)
st3 = st(fluido, {'P': P[2], 'H': h3 })

Definindo estados a partir da equação de eficiência isentrópica para bombas

$$ \eta_{pump} = \frac{h_{ns} - h_{n-1}}{h_{n} - h_{n-1}} $$

Estado 6

$$ st_{6s}: st_{6s}(P_{6}, S_{5}) $$
$$ h_{6} = h_{5} + \frac{h_{6s} - h_{5}}{\eta_{pump}} $$
$$ st_{6}: st_{6}(P_{6}, h_{6}) $$

In [110]:
st6s = st(fluido, {'P': P[5], 'S': st5.s })  
h6 = st5.h + (st6s.h - st5.h)/n_pump
st6 = st(fluido, {'P': P[5], 'H': h6 })

Estado 8

$$ st_{8s}: st_{8s}(P_{8}, S_{7}) $$
$$ h_{8} = h_{7} + \frac{h_{8s} - h_{7}}{\eta_{pump}} $$
$$ st_{8}: st_{8}(P_{8}, h_{8}) $$

In [111]:
st8s = st(fluido, {'P': P[7], 'S': st7.s })  ###
h8 = st7.h + (st8s.h - st7.h)/n_pump
st8 = st(fluido, {'P': P[7], 'H': h8 })

Estado 10

$$ st_{10s}: st_{10s}(P_{10}, S_{9}) $$
$$ h_{10} = h_{9} + \frac{h_{10s} - h_{9}}{\eta_{pump}} $$
$$ st_{10}: st_{10}(P_{10}, h_{10}) $$

In [112]:
st10s = st(fluido, {'P': P[9], 'S': st9.s}) 
h10 = st9.h + (st10s.h - st9.h)/n_pump
st10 = st(fluido, {'P': P[9], 'H': h10 })

## Obtendo valores de temperatura, pressão, entalpia, entropia e titulo para todos os estados ##

In [113]:
estds = st1, st2, st3, st4, st5, st6, st7, st8, st9, st10, st11, st12

temperatura, pressao, entalpia, entropia, titulo, estados = [], [], [], [], [], arange(1, len(estds)+1, 1)

for i in estds: 
    temperatura.append(i.T)
    pressao.append(i.p)
    entalpia.append(i.h)
    entropia.append(i.s)
    if absolute(round(i.Q, 2)) == 1:
        titulo.append('Vap. Sat.')
    elif i.Q == 0:
        titulo.append('Liq. Sat.')
    else:
        titulo.append(i.Q)

Definindo dicionário de dados dos estados e criando tabela

In [114]:
data_st = {
    'Estado': estados,
    'Temperatura [ºC]': temperatura,
    'Pressão [kPa]': pressao,
    'Entalpia [kJ/kg]': entalpia,
    'Entropia [kJ/kg/K]': entropia,
    'Titulo [-]': titulo
}

df_st = DataFrame(data_st).style.hide(axis='index').format(precision=2)

## Resultados ##

In [115]:
display(df_st)

Estado,Temperatura [ºC],Pressão [kPa],Entalpia [kJ/kg],Entropia [kJ/kg/K],Titulo [-]
1,758.15,6495.00,3381.20,6.79,Vap. Sat.
2,516.42,900.00,2932.08,6.95,Vap. Sat.
3,400.56,250.00,2714.68,7.05,Vap. Sat.
4,324.15,12978.07,224.65,0.71,Vap. Sat.
5,603.87,12978.07,1530.64,3.56,Liq. Sat.
6,400.56,250.00,1279.66,3.47,0.34
7,383.15,250.00,461.49,1.42,Vap. Sat.
8,383.90,6495.00,469.21,1.42,Vap. Sat.
9,383.15,250.00,461.49,1.42,Vap. Sat.
10,383.23,900.00,462.30,1.42,Vap. Sat.


## Trabalho, calor e eficiência ##

$$ \omega_{pump, in} = \eta_{pump}((h_{6} - h_{5}) + (h_{10} - h_{9}) + (h_{8} - h_{7}))  $$
$$ \omega_{turb, out} = \eta_{turb}((h_{2} - h_{1}) + (h_{3} - h_{2}) + (h_{4} - h_{3}))  $$
$$ q_{in} = h_{1} - h_{8} $$
$$ q_{out} = h_{4} - h_{5} $$
$$\eta_{th} = 1 - \frac{q_{out}}{q_{in}}$$

Considerando a turbina em regime permanente ($ V_{in} = V{out} $) temos que o trabalho realizado é dado pela equação:

$$ \dot{W} = \frac{dm}{dt}\omega_{turb, out} = V_{1} \omega_{turb, out}$$

In [116]:
w_pump_in = n_pump*((st5.h - st6.h) + (st9.h - st10.h) + (st7.h - st8.h))  
w_turb_out = n_turb*((st1.h - st2.h) + (st2.h - st3.h) + (st3.h - st4.h))
q_in = st1.h - st8.h
q_out = st5.h - st4.h
n_th = 100*( 1 - (q_out/q_in))
W = V[0]*w_turb_out

Definindo dicionário de dados de trabalho, calor e eficiêcia e criando tabela

In [117]:
data = {
    'w_pump_in [kJ/kg]': [w_pump_in],
    'w_turb_out [kJ/kg]': w_turb_out,
    'q_in [kJ/kg]': q_in,
    'q_out [kJ/kg]': q_out,
    'W [kW]': W,
    'η_th [%]': n_th
}

df = DataFrame(data).style.hide(axis='index').format(precision=2)

## Resultados ##

In [118]:
display(df)

w_pump_in [kJ/kg],w_turb_out [kJ/kg],q_in [kJ/kg],q_out [kJ/kg],W [kW],η_th [%]
206.09,2683.07,2911.99,1305.99,74857.53,55.15


Fazer a modelagem com condensador para que sua temperatura de condensação 
varie conforme a vazão de condensado. Mostrar essa variação.

![Condensador](img/Condensador.jpeg "Condensador")

Desenvolver a análise de primeira lei para o esquema de caldeira abaixo. 
Considerar que este sistema está inserido dentro do ciclo de potência já desenvolvido 
(Fig. 1).


![Caldeira](img/Caldeira.jpeg "Caldeira")